# 4.1 기존 시스템에 머신러닝을 통합하는 과정

1. 비즈니스 문제를 머신러닝 문제로 정의한다.
2. 논문을 중심으로 유사한 문제들을 조사한다.
3. 머신러닝을 사용하지 않는 방법은 없는지 검토한다.
4. **시스템을 설계**를 고려한다.
5. 특징량, 훈련 데이터와 **로그를 설계**한다.
6. 실제 데이터 수집과 전처리를 한다.
7. 탐색적 데이터 분석과 알고리즘을 선택한다.
8. 학습을 수행하고 파라미터를 튜닝한다.
9. 시스템에 통합한다.
10. 모델 예측 성능 및 비즈니스 지표를 모니터링 한다.

---

# 4.2 시스템 설계

### 4.2.1 혼동하기 쉬운 '배치 처리'와 '배치 학습'

- **배치 처리**: 어떤 처리를 일괄로 하는 것
- **실시간 처리**: 실시간으로 전송되는 데이터를 순차적으로 처리하는 것
- **일괄 학습(배치 학습)**: 새로운 가중치를 계산하는 데 훈련 데이터를 모두 사용.
    > SUM = w_1 + w_2 + w_3 + ... + w_100  
    w_target = sum / 100
- **순차 학습(온라인 학습)**: 훈련 데이터를 하나씩 입력하면서 새로운 가중치 계산.
    > sum = 0  
    cnt = 0  
    while has_weight():  
    &nbsp;&nbsp;&nbsp;&nbsp;w_tmp = get_weight()  
    &nbsp;&nbsp;&nbsp;&nbsp;sum += w_temp  
    &nbsp;&nbsp;&nbsp;&nbsp;cnt += 1  
    w_target = sum / cnt
    
    
> 일괄 학습과 순차 학습의 차이: 학습 도중 최적화 하는 방법의 차이와 한 번에 다루는 데이터 덩어리 크기의 차이.

- 배치 처리를 적용한 학습 패턴 3가지
    1. 배치 처리로 학습과 예측, 예측 결과를 DB에 저장하고 서비스 제공
    2. 배치 처리로 학습, 실시간 처리로 예측, 예측 결과를 API를 통해 제공
    3. 배치 처리로 학습, 최종단 클라이언트에서 실시간 처리로 예측
    4. 실시간 처리로 학습, 예측, 서비스를 제공
    
### 4.2.2 배치 처리로 학습과 예측, 예측 결과를 DB에 저장하고 서비스를 제공

- 우선 일괄 학습한 다음 배치 처리로 예측을 수행하고 예측 결과를 DB에 저장
    - DB를 통해 결과를 공유하기 때문에 웹 애플리케이션과 머신러닝을 수행하는 배치 시스템을 서로 다른 언어로 구현 가능
    - 예측 처리에 다소 시간이 걸리더라도 애플리케이션에서의 응답에 영향을 미치지 않음
- 특징
    - 예측에 필요한 정보는 예측 배치 처리를 실행할 때만 필요하다.
    - 이벤트(사용자의 웹 페이지 방문 등)에 대한 예측 결과를 즉시 반환하지 않아도 된다.
- 예측 실행 빈도가 적어도 수 시간 간격이어도 큰 문제가 없는 대상 또는 결과에 적합

![패턴1](./images/system1.png)

- 학습 단계에서는 로그 또는 사용자 정보에서 특징을 추출하고 모델을 일괄 학습한다. 이렇게 학습된 모델을 직렬화 하여 저장소제 저장하고 예측 단계에서 활용한다.
![패턴1](./images/system1-1.png)

- 특징 추출기를 이용해 DB 데이터에서 특징량을 추출해 예측을 수행한다. 예측 결과는 웹 애플리케이션이 사용할 수 있는 형태로 가공해 DB에 저장한다.
- 다른 패턴보다 예측에 사용할 수 있는 시간이 여유롭지만 예측 대상 컨텐츠가 늘어날수록 처리 시간도 늘어난다.
![패턴1](./images/system1-2.png)

### 4.2.3 배치로 처리로 학습, 실시간 처리로 예측, 예측 결과를 API 경유로 제공

- 예측 처리를 별도의 API 서버에서 제공하는 패턴
- HTTP나 RPC 요청의 응답으로 예측 결과를 전달하는 API 서버를 둔다.
![패턴2](./images/system2.png)

- 특징
    - 웹 애플리케이션과 머신러닝을 구현하는 언어가 서로 다르다.
    - 웹 애플리케이션에서 이벤트가 발생하면 예측이 실시간으로 수행된다.
- 프로토타이핑 주기가 짧아지나 시스템 규모가 커진다.
- 머신러닝 부분과 웹 애플리케이션의 결합이 느슨해져 여러 모델로 A/B 테스트를 하기 유리하다.
- 첫 번째 패턴에 비해 네트워크 지연이 크다.

### 4.2.4 배치 처리로 학습 + 예측 결과를 DB에 저장하고 사용(예측을 배치 처리)

- 웹 애플리케이션에 적용하기 가장 유리한 패턴
- 웹 애플리케이션과 머신러닝 부분을 서로 다른 언어로 구현해도 된다.
- 예측 처리에 다소 시간이 걸려도 웹 애플리케이션의 응답 시간에 영향을 주지 않는다.
- 특징
    - 예측에 필요한 정보는 예측 배치를 실행할 때만 필요하다.
    - (사용자의 웹 페이지 요청 같은) 이벤트에 대해 예측 결과를 즉각 알려주지 않아도 된다.
- 필요한 예측 빈도가 적을 때(하루 한 번 혹은 수시간 간격) 적용
![패턴3](./images/system3.png)

- 학습 단계
![패턴3](./images/system3-1.png)

- 학습 배치를 실행하는 간격은 예측을 실행하는 간격보다 길게 설정한다.
- 예측 단계에서는 학습 배치에서 만든 모델을 사용하고 예측 결과는 웹 애플리케이션이 사용할 수 있는 형태로 가공하여 DB에 저장한다.
![패턴3](./images/system3-2.png)

- 예측에 사용할 수 있는 시간이 다른 패턴보다 여유있다.
- 처리 시간이 컨텐츠 양에 비례해 늘어난다. 특히 자주 재학습을 하는 경우 또는 특징이나 알고리즘을 바꿔가며 모델을 여러개 학습하는 경우 주의 필요
- 병렬 작업 또는 분산 환경에서 빨라질 수 있다.

### 4.2.5 실시간 처리로 학습

- 밴디트 알고리즘 등 일부 알고리즘이나 실시간 추천이 필요한 경우 실시간 학습이 필요
- 모델을 비교적 짧은 간격으로 갱신해야 한다면 그 간격 동안 쌓인 데이터만을 배치 처리로 학습하되, 최적화에는 추가 학습이 가능한 미니배치 학습을 적용하는 것이 좋다.
- Oryx 프레임워크나 주바투스<sup>Jubatus</sup> 프레임워크 참고

### 4.2.6 각 패턴의 특성

| 패턴 | 일괄 학습 + 직접 예측 | 일괄 학습 + API 서버 | 일괄 학습 + DB | 실시간 |
|-----|------------------|-------------------|--------------|------|
| 예측 시점 | 요청 시        | 요청 시            | 배치           | 요청 시|
| 예측 결과 제공 경로 | 프로세스 내 API 경유 | REST API 경유 | 공유 DB 경유 | 메시지 큐 경유 |
| 예측 요청의 응답 지연시간 | ○ | ○ | ◎ | ◎ |
| 신규 데이터 추가부터 예측 결과까지 걸리는 시간 | 짧음 | 짧음 | 김 | 짧음|
| 한 건의 예측을 처리하는 시간 | 짧음 | 짧음 | 김 | 짧음 |
| 웹 애플리케이션과의 결합도 | 높음 | 낮음 | 낮음 | 낮음 |
| 두 시스템의 언어 구현 | 같아야 함 | 무관 | 무관 | 무관 |

- 이 중 한 패턴을 선택하기 위한 두 가지 조건
    - 웹 애플리케이션과 별도로 좋은 머신러닝 라이브러리를 갖춘 언어로 개발되어야 함.
    - 데이터 획득부터 예측 결과까지의 주기가 짧아야 한다.


---

# 4.3 로그 설계

- 지도 학습의 경우 웹 애플리케이션 로그나 사용자 클릭 로그 등을 수집하여 특징을 추출
- 로그 설계는 특징을 결정하기 위한 핵심
- 로그에 없는 값을 만들어내기 보다 미리 로그에 포함시키는 것이 중요

### 4.3.1 특징과 훈련 데이터에 사용되는 정보

1. 사용자 정보: 사용자가 가입할 때 제공하는 여러가지 속성 정보
2. 컨텐츠 정보: 블로그 포스팅이나 상품 정보 등. 보통 RDBMS에 저장
3. 사용자 행동 로그: 사용자의 접속 또는 발생시킨 이벤트 정보. 양이 많아 객체 저장소, 분산 RDBMS, 하둡 파일시스템 등에 저장

### 4.3.2 로그 저장하기

- 저장 방법
    - 분산 RDBMS에 저장
    - 하둡 클러스터의 분산 파일시스템(HDSF)에 저장
    - 객체 저장소에 저장
- SQL로 질의 할 수 있도록 준비가 필요
- 클라우드 서비스를 이용하여 비용을 절감할 수 있음

### 4.3.3 로그 설계 시의 주의점

- 앞으로 필요할지도 모르는 정보는 미리 로그에 포함시키는 것이 좋음
- 현재 수집 중인 로그에서 훈련 데이터를 만들 수 있는지 살피는 관점 필요
- 시스템 개발 및 운영 주체와 분석 주체가 분리되어 있는 경우 데이터 유실이 발생할 수 있다.
- 로그 포맷의 변경에 유의해야 한다.
- 대규모 데이터를 처리할 때는 가능한 한 데이터를 로컬 머신으로 내려받지 않는 수단을 마련해야 한다.

---

# 4.4 정리

- 일괄 학습을 통해 모델을 얻고 이 모델이 예측한 결과를 받는 4가지 패턴
    - 배치 처리로 학습 + 예측 결과를 웹 애플리케이션에서 직접 산출(예측을 실시간 처리)
    - 배치 처리로 학습 + 예측 결과를 API를 통해 사용(예측을 실시간 처리)
    - 배치 처리로 학습 + 예측 결과를 DB에 저장하고 사용(예측을 배치 처리)
    - 실시간 처리로 학습
- 로그 설계에 맞춰 특징이나 훈련 데이터를 꾸리는 것도 중요하며, 이런 경우 가급적이면 재적업할 일이 적어지도록 설계해야 함